In [1]:
#python packages
import random
import time
import operator
import numpy as np
# deap package
import evalGP
import gp_restrict
from deap import base, creator, tools, gp
# fitness function
from fitnessEvaluation import evalAccuracy, evalAccuracy_multiple
from STGPdataType import Img, Int1, Int2, Int3, Region, Float_point, Mask  # defined by author
import functionSet as fs

dataSetName='data/Sprat'
randomSeeds=12

x_train=np.load(dataSetName+'_X_train.npy')
y_train=np.load(dataSetName+'_y_train.npy')
x_validation=np.load(dataSetName+'_X_val.npy')
y_validation=np.load(dataSetName+'_y_val.npy')
x_test=np.load(dataSetName+'_X_test.npy')
y_test=np.load(dataSetName+'_y_test.npy')

#parameters:
population=100
generation=5
cxProb=0.8
mutProb=0.19
elitismProb=0.01
totalRuns = 1
initialMinDepth=2
initialMaxDepth=6
maxDepth=6
image_width, image_height = 445, 590

##GP
pset = gp.PrimitiveSetTyped('MAIN',[Img], Mask, prefix='Raw')
pset.addPrimitive(fs.ADD, [Region,Region], Mask, name='sum')#'sum'
pset.addPrimitive(fs.ADD2, [Region,Region], Mask, name='sum2')#'sum2'

pset.addPrimitive(fs.weighted_sum, [Region, Region, Float_point], Mask, name='Weight_sum')#'weighted_sum' 


pset.addPrimitive(fs.walk, [Img, Int2, Int3], Region, name='walk')
pset.addPrimitive(fs.Snakes, [Img, Int2, Int3], Region, name='Snakes')
pset.addPrimitive(fs.waterShed, [Img], Region, name='watershed')
pset.addPrimitive(fs.chanVese, [Img], Region, name='chan_vese')
# pset.addPrimitive(fs.K_means_clustering, [Img], Region, name='K-means') 
pset.addPrimitive(fs.Canny_edge, [Img], Region, name='Canny')
pset.addPrimitive(fs.Thresholding, [Img, Int1], Region, name='Thresh')
# Terminals
pset.renameArguments(ARG0='grey')
pset.addEphemeralConstant('a_Thres', lambda: 0.1 * random.randint(1,9), Int1)
pset.addEphemeralConstant('b_Sigma', lambda: 0.1 * random.randint(1,9), Int2)
pset.addEphemeralConstant('c_Beta', lambda: random.randint(5, 20), Int3)
pset.addEphemeralConstant('d_randomDouble', lambda: round(random.random(), 2), Float_point)

##
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("expr", gp_restrict.genHalfAndHalfMD, pset=pset, min_=initialMinDepth, max_=initialMaxDepth)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

toolbox.register("evaluate", evalAccuracy,toolbox,x_train=x_train,y_train=y_train)
toolbox.register("validation", evalAccuracy,toolbox,x_train=x_validation,y_train=y_validation)
toolbox.register("select", tools.selTournament,tournsize=7)
toolbox.register("selectElitism", tools.selBest)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp_restrict.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)
toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=maxDepth))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=maxDepth))

def GPMain(randomSeeds):

    random.seed(randomSeeds)
    pop = toolbox.population(population)
    hof = tools.HallOfFame(1)
    log = tools.Logbook()
    stats_fit = tools.Statistics(key=lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(key=len)
    mstats = tools.MultiStatistics(fitness=stats_fit,size=stats_size)
    mstats.register("avg", np.mean)
    mstats.register("std", np.std)
    mstats.register("min", np.min)
    mstats.register("max", np.max)
    log.header = ["gen", "evals"] + mstats.fields

    pop, log, hof2= evalGP.eaSimple(pop, toolbox, cxProb, mutProb,elitismProb, generation,
                    stats=mstats, halloffame=hof, verbose=True)

    return pop,log, hof,hof2

if __name__ == "__main__":

    beginTime = time.process_time()
    pop, log, hof, hof2 = GPMain(randomSeeds)
    endTime = time.process_time()
    trainTime = endTime - beginTime

#     testResults = evalAccuracy(toolbox, hof2[0], x_test, y_test)
    f1_score,accuracy, recall, precision = evalAccuracy_multiple(toolbox, hof2[0], x_test, y_test)
    
    testTime = time.process_time() - endTime

    print('Best individual ', hof[0])
    print('Test results  ', testResults)
    print('Train time  ', trainTime)
    print('Test time  ', testTime)
    
    print('f1_score', f1_score)
    print('accuracy', accuracy)
    print('recall', recall)
    print('precision', precision)
    
    print('End')

KeyboardInterrupt: 

In [2]:
#python packages
import random
import time
import operator
import numpy as np
# deap package
import evalGP
import gp_restrict
from deap import base, creator, tools, gp
# fitness function
from fitnessEvaluation import evalAccuracy, evalAccuracy_multiple
from STGPdataType import Img, Int1, Int2, Int3, Region, Float_point, Mask  # defined by author
import functionSet as fs

dataSetName='data/Gilt'
randomSeeds=12

x_train=np.load(dataSetName+'_X_train.npy')
y_train=np.load(dataSetName+'_y_train.npy')
x_validation=np.load(dataSetName+'_X_val.npy')
y_validation=np.load(dataSetName+'_y_val.npy')
x_test=np.load(dataSetName+'_X_test.npy')
y_test=np.load(dataSetName+'_y_test.npy')

#parameters:
population=100
generation=5
cxProb=0.8
mutProb=0.19
elitismProb=0.01
totalRuns = 1
initialMinDepth=2
initialMaxDepth=6
maxDepth=6
image_width, image_height = 445, 590

##GP
pset = gp.PrimitiveSetTyped('MAIN',[Img], Mask, prefix='Raw')
pset.addPrimitive(fs.ADD, [Region,Region], Mask, name='sum')#'sum'
pset.addPrimitive(fs.ADD2, [Region,Region], Mask, name='sum2')#'sum2'

pset.addPrimitive(fs.weighted_sum, [Region, Region, Float_point], Mask, name='Weight_sum')#'weighted_sum' 


pset.addPrimitive(fs.walk, [Img, Int2, Int3], Region, name='walk')
pset.addPrimitive(fs.Snakes, [Img, Int2, Int3], Region, name='Snakes')
pset.addPrimitive(fs.waterShed, [Img], Region, name='watershed')
pset.addPrimitive(fs.chanVese, [Img], Region, name='chan_vese')
# pset.addPrimitive(fs.K_means_clustering, [Img], Region, name='K-means') 
pset.addPrimitive(fs.Canny_edge, [Img], Region, name='Canny')
pset.addPrimitive(fs.Thresholding, [Img, Int1], Region, name='Thresh')
# Terminals
pset.renameArguments(ARG0='grey')
pset.addEphemeralConstant('a_Thres', lambda: 0.1 * random.randint(1,9), Int1)
pset.addEphemeralConstant('b_Sigma', lambda: 0.1 * random.randint(1,9), Int2)
pset.addEphemeralConstant('c_Beta', lambda: random.randint(5, 20), Int3)
pset.addEphemeralConstant('d_randomDouble', lambda: round(random.random(), 2), Float_point)

##
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("expr", gp_restrict.genHalfAndHalfMD, pset=pset, min_=initialMinDepth, max_=initialMaxDepth)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

toolbox.register("evaluate", evalAccuracy,toolbox,x_train=x_train,y_train=y_train)
toolbox.register("validation", evalAccuracy,toolbox,x_train=x_validation,y_train=y_validation)
toolbox.register("select", tools.selTournament,tournsize=7)
toolbox.register("selectElitism", tools.selBest)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp_restrict.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)
toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=maxDepth))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=maxDepth))

def GPMain(randomSeeds):

    random.seed(randomSeeds)
    pop = toolbox.population(population)
    hof = tools.HallOfFame(1)
    log = tools.Logbook()
    stats_fit = tools.Statistics(key=lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(key=len)
    mstats = tools.MultiStatistics(fitness=stats_fit,size=stats_size)
    mstats.register("avg", np.mean)
    mstats.register("std", np.std)
    mstats.register("min", np.min)
    mstats.register("max", np.max)
    log.header = ["gen", "evals"] + mstats.fields

    pop, log, hof2= evalGP.eaSimple(pop, toolbox, cxProb, mutProb,elitismProb, generation,
                    stats=mstats, halloffame=hof, verbose=True)

    return pop,log, hof,hof2

if __name__ == "__main__":

    beginTime = time.process_time()
    pop, log, hof, hof2 = GPMain(randomSeeds)
    endTime = time.process_time()
    trainTime = endTime - beginTime

#     testResults = evalAccuracy(toolbox, hof2[0], x_test, y_test)
    f1_score,accuracy, recall, precision = evalAccuracy_multiple(toolbox, hof2[0], x_test, y_test)
    
    testTime = time.process_time() - endTime

    print('Best individual ', hof[0])
    print('Test results  ', testResults)
    print('Train time  ', trainTime)
    print('Test time  ', testTime)
    
    print('f1_score', f1_score)
    print('accuracy', accuracy)
    print('recall', recall)
    print('precision', precision)
    
    print('End')

Exception: Ephemerals with different functions should be named differently, even between psets.

In [ ]:
#python packages
import random
import time
import operator
import numpy as np
# deap package
import evalGP
import gp_restrict
from deap import base, creator, tools, gp
# fitness function
from fitnessEvaluation import evalAccuracy, evalAccuracy_multiple
from STGPdataType import Img, Int1, Int2, Int3, Region, Float_point, Mask  # defined by author
import functionSet as fs

dataSetName='data/Hourse'
randomSeeds=12

x_train=np.load(dataSetName+'_X_train.npy')
y_train=np.load(dataSetName+'_y_train.npy')
x_validation=np.load(dataSetName+'_X_val.npy')
y_validation=np.load(dataSetName+'_y_val.npy')
x_test=np.load(dataSetName+'_X_test.npy')
y_test=np.load(dataSetName+'_y_test.npy')

#parameters:
population=100
generation=5
cxProb=0.8
mutProb=0.19
elitismProb=0.01
totalRuns = 1
initialMinDepth=2
initialMaxDepth=6
maxDepth=6
image_width, image_height = 445, 590

##GP
pset = gp.PrimitiveSetTyped('MAIN',[Img], Mask, prefix='Raw')
pset.addPrimitive(fs.ADD, [Region,Region], Mask, name='sum')#'sum'
pset.addPrimitive(fs.ADD2, [Region,Region], Mask, name='sum2')#'sum2'

pset.addPrimitive(fs.weighted_sum, [Region, Region, Float_point], Mask, name='Weight_sum')#'weighted_sum' 


pset.addPrimitive(fs.walk, [Img, Int2, Int3], Region, name='walk')
pset.addPrimitive(fs.Snakes, [Img, Int2, Int3], Region, name='Snakes')
pset.addPrimitive(fs.waterShed, [Img], Region, name='watershed')
pset.addPrimitive(fs.chanVese, [Img], Region, name='chan_vese')
# pset.addPrimitive(fs.K_means_clustering, [Img], Region, name='K-means') 
pset.addPrimitive(fs.Canny_edge, [Img], Region, name='Canny')
pset.addPrimitive(fs.Thresholding, [Img, Int1], Region, name='Thresh')
# Terminals
pset.renameArguments(ARG0='grey')
pset.addEphemeralConstant('a_Thres', lambda: 0.1 * random.randint(1,9), Int1)
pset.addEphemeralConstant('b_Sigma', lambda: 0.1 * random.randint(1,9), Int2)
pset.addEphemeralConstant('c_Beta', lambda: random.randint(5, 20), Int3)
pset.addEphemeralConstant('d_randomDouble', lambda: round(random.random(), 2), Float_point)

##
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("expr", gp_restrict.genHalfAndHalfMD, pset=pset, min_=initialMinDepth, max_=initialMaxDepth)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

toolbox.register("evaluate", evalAccuracy,toolbox,x_train=x_train,y_train=y_train)
toolbox.register("validation", evalAccuracy,toolbox,x_train=x_validation,y_train=y_validation)
toolbox.register("select", tools.selTournament,tournsize=7)
toolbox.register("selectElitism", tools.selBest)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp_restrict.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)
toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=maxDepth))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=maxDepth))

def GPMain(randomSeeds):

    random.seed(randomSeeds)
    pop = toolbox.population(population)
    hof = tools.HallOfFame(1)
    log = tools.Logbook()
    stats_fit = tools.Statistics(key=lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(key=len)
    mstats = tools.MultiStatistics(fitness=stats_fit,size=stats_size)
    mstats.register("avg", np.mean)
    mstats.register("std", np.std)
    mstats.register("min", np.min)
    mstats.register("max", np.max)
    log.header = ["gen", "evals"] + mstats.fields

    pop, log, hof2= evalGP.eaSimple(pop, toolbox, cxProb, mutProb,elitismProb, generation,
                    stats=mstats, halloffame=hof, verbose=True)

    return pop,log, hof,hof2

if __name__ == "__main__":

    beginTime = time.process_time()
    pop, log, hof, hof2 = GPMain(randomSeeds)
    endTime = time.process_time()
    trainTime = endTime - beginTime

#     testResults = evalAccuracy(toolbox, hof2[0], x_test, y_test)
    f1_score,accuracy, recall, precision = evalAccuracy_multiple(toolbox, hof2[0], x_test, y_test)
    
    testTime = time.process_time() - endTime

    print('Best individual ', hof[0])
    print('Test results  ', testResults)
    print('Train time  ', trainTime)
    print('Test time  ', testTime)
    
    print('f1_score', f1_score)
    print('accuracy', accuracy)
    print('recall', recall)
    print('precision', precision)
    
    print('End')